## Install necessary libraries needed

In [ ]:
pip install psycopg2 faker

<h2>1. Import Necessary Libraries needed</h2>

In [1]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

<h2>Note: I created a .py file that contains my aiven credentials</h2>

In [4]:
import aiven_credentials

In [6]:
my_user = aiven_credentials.user
my_password = aiven_credentials.password
my_host = aiven_credentials.host
my_port = aiven_credentials.port

<h2>2. Create a connection to your database using aiven credential details</h2>

In [11]:
conn = psycopg2.connect(
    dbname="defaultdb",
    user=my_user[0],
    password=my_password[0],
    host = my_host[0],
    port=my_port
)
cursor = conn.cursor()
fake = Faker()

<h2>3. Populated your tables using dummy data</h2>

In [13]:
# Insert instructors
instructor_ids = []
for _ in range(5):
    fn, ln = fake.first_name(), fake.last_name()
    email = f"{fn.lower()}.{ln.lower()}@university.edu"
    cursor.execute("""
        INSERT INTO instructors (first_name, last_name, email)
        VALUES (%s, %s, %s) RETURNING instructor_id;
    """, (fn, ln, email))
    instructor_ids.append(cursor.fetchone()[0])

# Insert courses
course_ids = []
for _ in range(10):
    course_name = fake.job()
    desc = fake.text(max_nb_chars=100)
    instructor_id = random.choice(instructor_ids)
    cursor.execute("""
        INSERT INTO courses (course_name, course_description, instructor_id)
        VALUES (%s, %s, %s) RETURNING course_id;
    """, (course_name, desc, instructor_id))
    course_ids.append(cursor.fetchone()[0])

# Insert students
student_ids = []
for _ in range(30):
    fn, ln = fake.first_name(), fake.last_name()
    email = f"{fn.lower()}.{ln.lower()}@student.edu"
    dob = fake.date_of_birth(minimum_age=18, maximum_age=35)
    cursor.execute("""
        INSERT INTO students (first_name, last_name, email, date_of_birth)
        VALUES (%s, %s, %s, %s) RETURNING student_id;
    """, (fn, ln, email, dob))
    student_ids.append(cursor.fetchone()[0])

# Insert enrollments
grades = ['A', 'B', 'C', 'D', 'F']
for _ in range(50):
    student_id = random.choice(student_ids)
    course_id = random.choice(course_ids)
    enroll_date = fake.date_between(start_date='-1y', end_date='today')
    grade = random.choice(grades)
    cursor.execute("""
        INSERT INTO enrollments (student_id, course_id, enrollment_date, grade)
        VALUES (%s, %s, %s, %s);
    """, (student_id, course_id, enroll_date, grade))

# Commit and close
conn.commit()
cursor.close()
conn.close()

print("Database populated successfully.")

Database populated successfully.
